In [1]:
import torch, detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Tue_May__3_18:49:52_PDT_2022
Cuda compilation tools, release 11.7, V11.7.64
Build cuda_11.7.r11.7/compiler.31294372_0
torch:  2.0 ; cuda:  cu117
detectron2: 0.6


In [2]:
import os
import sys
import logging
import argparse

pth = '/'.join(sys.path[0].split('/')[:-1])
sys.path.insert(0, pth)

from pprint import pprint
from PIL import Image
from copy import deepcopy
import numpy as np
np.random.seed(1)

home_dir = os.path.abspath(os.getcwd()+"/../")
sys.path.append(home_dir)
print(home_dir)

import warnings
warnings.filterwarnings(action='ignore')
# logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

import torch
from torchvision import transforms
from torch.nn import functional as F

from detectron2.data import MetadataCatalog
from detectron2.utils.colormap import random_color
from detectron2.structures import Boxes, ImageList, Instances, BitMasks, BoxMode

from hdecoder.BaseModel import BaseModel
from hdecoder import build_model

from utils.arguments import load_opt_command
from utils.misc import hook_metadata, hook_switcher, hook_opt
from utils.distributed import init_distributed
from utils.arguments import load_opt_from_config_files, load_config_dict_to_opt

/home/djjin/Mygit/X-Decoder


Invalid MIT-MAGIC-COOKIE-1 key

In [3]:
parser = argparse.ArgumentParser(description='Pretrain or fine-tune models for NLP tasks.')
parser.add_argument('--command', default="evaluate", help='Command: train/evaluate/train-and-evaluate')
parser.add_argument('--conf_files', nargs='+', help='Path(s) to the config file(s).')
parser.add_argument('--user_dir', help='Path to the user defined module for tasks (models, criteria), optimizers, and lr schedulers.')
parser.add_argument('--config_overrides', nargs='*', help='Override parameters on config with a json style string, e.g. {"<PARAM_NAME_1>": <PARAM_VALUE_1>, "<PARAM_GROUP_2>.<PARAM_SUBGROUP_2>.<PARAM_2>": <PARAM_VALUE_2>}. A key with "." updates the object in the corresponding nested dict. Remember to escape " in command line.')
parser.add_argument('--overrides', help='arguments that used to override the config file in cmdline', nargs=argparse.REMAINDER)

cmdline_args = parser.parse_args('')
# cmdline_args.conf_files = [os.path.join(home_dir, "configs/xdecoder/svlp_focalt_lang.yaml")]
cmdline_args.conf_files = [os.path.join(home_dir, "configs/xdecoder/vcoco.yaml")]
cmdline_args.overrides = ['WEIGHT', 'true', 'RESUME_FROM', '../checkpoints/xdecoder_focalt_best_openseg.pt'] 

opt = load_opt_from_config_files(cmdline_args.conf_files)

keys = [cmdline_args.overrides[idx*2] for idx in range(len(cmdline_args.overrides)//2)]
vals = [cmdline_args.overrides[idx*2+1] for idx in range(len(cmdline_args.overrides)//2)]
print(keys, vals)
vals = [val.replace('false', '').replace('False','') if len(val.replace(' ', '')) == 5 else val for val in vals]
types = []
for key in keys:
    key = key.split('.')
    ele = opt.copy()
    while len(key) > 0:
        ele = ele[key.pop(0)]
    types.append(type(ele))

config_dict = {x:z(y) for x,y,z in zip(keys, vals, types)}

load_config_dict_to_opt(opt, config_dict)
for key, val in cmdline_args.__dict__.items():
    if val is not None:
        opt[key] = val
opt = init_distributed(opt)

['WEIGHT', 'RESUME_FROM'] ['true', '../checkpoints/xdecoder_focalt_best_openseg.pt']


In [4]:
pretrained_pth = os.path.join(opt['RESUME_FROM'])
print(pretrained_pth)

../checkpoints/xdecoder_focalt_best_openseg.pt


In [5]:
model = BaseModel(opt, build_model(opt)).from_pretrained(pretrained_pth).train().cuda()

*UNLOADED* hoid_head.hoi_decoder.hopd_decoder.layers.0.linear1.bias, Model Shape: torch.Size([2048])
*UNLOADED* hoid_head.hoi_decoder.hopd_decoder.layers.0.linear1.weight, Model Shape: torch.Size([2048, 512])
*UNLOADED* hoid_head.hoi_decoder.hopd_decoder.layers.0.linear2.bias, Model Shape: torch.Size([512])
*UNLOADED* hoid_head.hoi_decoder.hopd_decoder.layers.0.linear2.weight, Model Shape: torch.Size([512, 2048])
*UNLOADED* hoid_head.hoi_decoder.hopd_decoder.layers.0.multihead_attn.in_proj_bias, Model Shape: torch.Size([1536])
*UNLOADED* hoid_head.hoi_decoder.hopd_decoder.layers.0.multihead_attn.in_proj_weight, Model Shape: torch.Size([1536, 512])
*UNLOADED* hoid_head.hoi_decoder.hopd_decoder.layers.0.multihead_attn.out_proj.bias, Model Shape: torch.Size([512])
*UNLOADED* hoid_head.hoi_decoder.hopd_decoder.layers.0.multihead_attn.out_proj.weight, Model Shape: torch.Size([512, 512])
*UNLOADED* hoid_head.hoi_decoder.hopd_decoder.layers.0.norm1.bias, Model Shape: torch.Size([512])
*UNLOAD

In [6]:
# t = []
# t.append(transforms.Resize(800, interpolation=Image.BICUBIC))
# transform = transforms.Compose(t)
# model.eval()
# model.model.backbone

# pixel_mean = torch.Tensor([123.675, 116.280, 103.530]).view(-1, 1, 1).cuda()
# pixel_std = torch.Tensor([58.395, 57.120, 57.375]).view(-1, 1, 1).cuda()

# num_layers = opt['MODEL']['BACKBONE']['FOCAL']['DEPTHS']
# out_indices = opt['MODEL']['BACKBONE']['FOCAL']['OUT_INDICES']
# focal_levels = opt['MODEL']['BACKBONE']['FOCAL']['FOCAL_LEVELS']
# num_layers, out_indices, focal_levels
# image_pth = '../images/animals.png'

# with torch.no_grad():
#     image_ori = Image.open(image_pth).convert('RGB')
#     width = image_ori.size[0]
#     height = image_ori.size[1]
#     image = transform(image_ori)
#     image = np.asarray(image)
#     image_ori = np.asarray(image_ori)
#     images = torch.from_numpy(image.copy()).permute(2,0,1).cuda()
#     batch_inputs = [{'image': images, 'height': height, 'width': width}]
#     # outputs = model.forward(batch_inputs)

#     images = [x["image"].to("cuda") for x in batch_inputs]
#     images = [(x - pixel_mean) / pixel_std for x in images]
    
#     images = ImageList.from_tensors(images, 32)
#     print(f"Image shape: {images.tensor.shape}")

#     ######################################################
#     ## Backbone
#     outputs = model.model.backbone(images.tensor)
#     ## Encoder
#     transformer_encoder_features, _ = model.model.hoid_head.encoder.forward_features(outputs)
#     print(transformer_encoder_features.shape)

In [7]:
pprint(opt)

{'CUDA': True,
 'DATALOADER': {'ASPECT_RATIO_GROUPING': False,
                'FILTER_EMPTY_ANNOTATIONS': False,
                'LOAD_PROPOSALS': False,
                'NUM_WORKERS': 1,
                'SAMPLER_TRAIN': 'TrainingSampler'},
 'DATASETS': {'CLASS_CONCAT': False,
              'PROPOSAL_FILES_TRAIN': [],
              'SIZE_DIVISIBILITY': 32,
              'TEST': ['vcoco_val'],
              'TRAIN': ['vcoco_train']},
 'DONT_LOAD_MODEL': True,
 'FIND_UNUSED_PARAMETERS': False,
 'FP16': False,
 'INPUT': {'PIXEL_MEAN': [123.675, 116.28, 103.53],
           'PIXEL_STD': [58.395, 57.12, 57.375]},
 'LOADER': {'JOINT': True, 'KEY_DATASET': 'coco'},
 'LOG_EVERY': 100,
 'MODEL': {'BACKBONE': {'FOCAL': {'DEPTHS': [2, 2, 6, 2],
                                  'DROP_PATH_RATE': 0.3,
                                  'DROP_RATE': 0.0,
                                  'EMBED_DIM': 96,
                                  'FOCAL_LEVELS': [3, 3, 3, 3],
                                

```sh
CUDA_VISIBLE_DEVICES=0 python entry.py train \
            --conf_files configs/xdecoder/segvlp_focalt_lang.yaml \
            --overrides \
            COCO.INPUT.IMAGE_SIZE 1024 \
            MODEL.DECODER.CAPTIONING.ENABLED True \
            MODEL.DECODER.RETRIEVAL.ENABLED True \
            MODEL.DECODER.GROUNDING.ENABLED True \
            MODEL.DECODER.CAPTIONING_WEIGHT 8 \
            MODEL.DECODER.RETRIEVAL_WEIGHT 8 \
            MODEL.DECODER.TOP_CAPTIONING_LAYERS 3 \
            MODEL.DECODER.TOP_RETRIEVAL_LAYERS 3 \
            MODEL.DECODER.TOP_GROUNDING_LAYERS 6 \
            COCO.TEST.BATCH_SIZE_TOTAL 1 \
            COCO.TRAIN.BATCH_SIZE_TOTAL 1 \
            COCO.TRAIN.BATCH_SIZE_PER_GPU 1 \
            VLP.TEST.BATCH_SIZE_TOTAL 2 \
            VLP.TRAIN.BATCH_SIZE_TOTAL 2 \
            VLP.TRAIN.BATCH_SIZE_PER_GPU 2 \
            MODEL.DECODER.HIDDEN_DIM 512 \
            MODEL.ENCODER.CONVS_DIM 512 \
            MODEL.ENCODER.MASK_DIM 512 \
            FP16 True \
            WEIGHT True \
            RESUME_FROM checkpoints/xdecoder_focalt_last.pt
```

In [8]:
from trainer import HDecoder_Trainer as Trainer
trainer = Trainer(opt)
trainer.train()

----------------
MPI Adapter data
----------------
environment info: no MPI
init method url: tcp://127.0.0.1:36873
world size: 1
local size: 1
rank: 0
local rank: 0
master address: 127.0.0.1
master port: 36873
----------------


Dataset length: 5400
num of train samples: 5400
epochs[     0] optim steps[1/5400] learning rate[default: 1.00000e-05] train loss[loss_obj_ce: 4.18997/4.18997, obj_class_error: 100.00000/100.00000, loss_verb_ce: 149.85196/149.85196, loss_sub_bbox: 0.57681/0.57681, loss_obj_bbox: 1.01425/1.01425, loss_sub_giou: 0.96354/0.96354, loss_obj_giou: 1.08783/1.08783, obj_cardinality_error: 99.00000/99.00000, loss_obj_ce_0: 3.92688/3.92688, loss_verb_ce_0: 169.41173/169.41173, loss_sub_bbox_0: 0.55769/0.55769, loss_obj_bbox_0: 1.01498/1.01498, loss_sub_giou_0: 0.94184/0.94184, loss_obj_giou_0: 1.08712/1.08712, obj_cardinality_error_0: 99.00000/99.00000, loss_obj_ce_1: 4.12633/4.12633, loss_verb_ce_1: 154.78200/154.78200, loss_sub_bbox_1: 0.55990/0.55990, loss_obj_bbox_1: 1.01971/1.01971, loss_sub_giou_1: 0.95249/0.95249, loss_obj_giou_1: 1.08928/1.08928, obj_cardinality_error_1: 99.00000/99.00000] items per batch[1] items per second[1.10] total items[1] mini batches[     0] memory[1973] epoch re

KeyboardInterrupt: 